In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\lenovo\Desktop\Deep Learning Project','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\lenovo\Desktop\Deep Learning Project','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxtPath

'C:\\Users\\lenovo\\Desktop\\Deep Learning Project\\deploy.prototxt'

In [4]:
weightsPath

'C:\\Users\\lenovo\\Desktop\\Deep Learning Project\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'C:\Users\lenovo\Desktop\Deep Learning Project\mobilenet_v2.model')

In [10]:
image=cv2.imread(r'C:\Users\lenovo\Desktop\Deep Learning Project\imagee.jpeg')

In [11]:
image

array([[[ 18,  18,  12],
        [ 16,  16,  10],
        [ 12,  13,   4],
        ...,
        [ 49,  54,  45],
        [ 43,  48,  39],
        [ 40,  45,  36]],

       [[ 18,  18,  12],
        [ 16,  16,  10],
        [ 12,  13,   4],
        ...,
        [ 51,  54,  45],
        [ 44,  50,  39],
        [ 44,  47,  38]],

       [[ 18,  18,  12],
        [ 16,  16,  10],
        [ 12,  13,   4],
        ...,
        [ 52,  53,  43],
        [ 48,  52,  40],
        [ 48,  49,  39]],

       ...,

       [[ 90, 157, 150],
        [ 90, 157, 148],
        [ 89, 158, 148],
        ...,
        [189, 202, 228],
        [190, 203, 229],
        [190, 203, 229]],

       [[ 89, 155, 150],
        [ 89, 156, 149],
        [ 88, 156, 149],
        ...,
        [187, 200, 226],
        [188, 201, 227],
        [188, 201, 227]],

       [[ 88, 154, 149],
        [ 88, 154, 149],
        [ 88, 156, 149],
        ...,
        [186, 199, 225],
        [186, 199, 225],
        [186, 199, 225]]

In [12]:
(h,w)=image.shape[:2]

In [13]:
(h,w)

(182, 277)

In [14]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [15]:
blob

array([[[[ -86.,  -88.,  -91., ...,  -56.,  -61.,  -64.],
         [ -86.,  -88.,  -91., ...,  -56.,  -61.,  -62.],
         [ -86.,  -88.,  -91., ...,  -54.,  -60.,  -60.],
         ...,
         [ -15.,  -15.,  -16., ...,   83.,   84.,   84.],
         [ -16.,  -16.,  -16., ...,   82.,   83.,   83.],
         [ -16.,  -16.,  -16., ...,   82.,   82.,   82.]],

        [[-159., -161., -164., ..., -124., -129., -132.],
         [-159., -161., -163., ..., -124., -129., -131.],
         [-159., -161., -164., ..., -124., -127., -130.],
         ...,
         [ -22.,  -21.,  -21., ...,   23.,   24.,   24.],
         [ -23.,  -22.,  -21., ...,   22.,   23.,   23.],
         [ -23.,  -23.,  -22., ...,   22.,   22.,   22.]],

        [[-111., -113., -118., ...,  -79.,  -84.,  -87.],
         [-111., -113., -118., ...,  -79.,  -84.,  -86.],
         [-111., -113., -118., ...,  -79.,  -84.,  -85.],
         ...,
         [  27.,   26.,   26., ...,  103.,  104.,  104.],
         [  26.,   26.,   

In [16]:
blob.shape

(1, 3, 300, 300)

In [17]:
net.setInput(blob)
detections=net.forward()

In [18]:
detections

array([[[[0.        , 1.        , 0.9567166 , ..., 0.16750354,
          0.598803  , 0.5967531 ],
         [0.        , 1.        , 0.21466264, ..., 0.27122858,
          0.37934142, 0.539935  ],
         [0.        , 1.        , 0.128831  , ..., 4.002771  ,
          4.83937   , 4.9860063 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [19]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
